# **Guided Project: Exploring eBay Car Sales Data**

# Introducing

**In this guided project, we'll work with a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website. You can find it [here](https://data.world/data-society/used-cars-data).**

The **aim** of this project is **to clean the dataset and perform some initial analysis on it**. While working on this project, you'll become familiar with some of the unique benefits the JupyterLab environment has to offer for pandas.

Let's start with importing libraries.

# Preparing for analysis

In [1]:
import numpy as np
import pandas as pd

Then we read the **autos.csv** file into dataframe **autos**:

In [2]:
autos = pd.read_csv("autos.csv", encoding = "latin1")

Let's explore our dataset:

In [3]:
print(autos.head(5))
autos.describe()

           dateCrawled                                               name  \
0  2016-03-26 17:47:46                   Peugeot_807_160_NAVTECH_ON_BOARD   
1  2016-04-04 13:38:56         BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik   
2  2016-03-26 18:57:24                         Volkswagen_Golf_1.6_United   
3  2016-03-12 16:58:10  Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...   
4  2016-04-01 14:38:50  Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...   

   seller offerType   price   abtest vehicleType  yearOfRegistration  \
0  privat   Angebot  $5,000  control         bus                2004   
1  privat   Angebot  $8,500  control   limousine                1997   
2  privat   Angebot  $8,990     test   limousine                2009   
3  privat   Angebot  $4,350  control  kleinwagen                2007   
4  privat   Angebot  $1,350     test       kombi                2003   

     gearbox  powerPS   model   odometer  monthOfRegistration fuelType  \
0    manuell      158  andere 

,yearOfRegistration,powerPS,monthOfRegistration,nrOfPictures,postalCode
count,50000.000000,50000.000000,50000.000000,50000.0,50000.000000
mean,2005.073280,116.355920,5.723360,0.0,50813.627300
std,105.712813,209.216627,3.711984,0.0,25779.747957
min,1000.000000,0.000000,0.000000,0.0,1067.000000
25%,1999.000000,70.000000,3.000000,0.0,30451.000000
50%,2003.000000,105.000000,6.000000,0.0,49577.000000
75%,2008.000000,150.000000,9.000000,0.0,71540.000000
max,9999.000000,17700.000000,12.000000,0.0,99998.000000


We can see that our dataset has 20 columns, but only 7 of them are the columns with dtype int or float. We can see, that we have in some positions the Nan values and probably we need in future to fix this things. Also we can see, that this dataset use camelcase instead of Python's preferred snakecase, which means we can't just replace spaces with underscores.

## Cleaning Column Names

Let's clean our columns from camelcase to snakecase, which Python preffered most:

In [4]:
print(autos.columns)

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')


Let's create a dictionary with keys, which represents the names of current columns, and the values are the future names of columns. Then we can change names of our columns:

In [5]:
dictionary_of_changes = {"dateCrawled" : "date_crawled", 
                         "offerType" : "offer_type", 
                         "vehicleType" : "type_vehicle", 
                         "yearOfRegistration" : "registration_year", 
                         'monthOfRegistration' : 'registration_month', 
                         "powerPS" : "power_ps", 
                         "fuelType" : "fuel_type", 
                         "notRepairedDamage" : "unrepaired_damage", 
                         "dateCreated" : "ad_created", 
                         "nrOfPictures" : "nr_of_pictures", 
                         "postalCode" : "postal_code", 
                         "lastSeen" : "last_seen",
                        }

def rename_columns(string):
    string.rename(dictionary_of_changes, axis = 1, inplace = True)
    return string

autos = rename_columns(autos)

Let's see the results:

In [6]:
print(autos.columns)
print(autos.head())

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'type_vehicle', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')
          date_crawled                                               name  \
0  2016-03-26 17:47:46                   Peugeot_807_160_NAVTECH_ON_BOARD   
1  2016-04-04 13:38:56         BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik   
2  2016-03-26 18:57:24                         Volkswagen_Golf_1.6_United   
3  2016-03-12 16:58:10  Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...   
4  2016-04-01 14:38:50  Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...   

   seller offer_type   price   abtest type_vehicle  registration_year  \
0  privat    Angebot  $5,000  control          bus               2004   
1  privat    Angebot  $8,500  control    limousine          

We have made this changes, because the snakecase are the common in Python syntax, and I think, that it will be comfortable  to  work with current names of columns in our future steps.

## Initial Exploration and Cleaning

Let's explore our dataset and look for columns where all or almost all values are the same and examples of numeric data stored as text which can be cleaned and converted.

In [7]:
autos.describe(include = "all")

,date_crawled,name,seller,offer_type,price,abtest,type_vehicle,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-04-02 11:37:04,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Let's explore some columns in details:

In [8]:
print(autos["seller"].value_counts())
print(autos["offer_type"].value_counts())

seller
privat        49999
gewerblich        1
Name: count, dtype: int64
offer_type
Angebot    49999
Gesuch         1
Name: count, dtype: int64


We can see that these columns have in common only one type, so we may drop it from our dataset or not include in our analyses.

Now let's see to potentially numeric columns:

In [9]:
print(autos["price"].unique())

print(autos["odometer"].unique())

print(autos["nr_of_pictures"].unique())

['$5,000' '$8,500' '$8,990' ... '$385' '$22,200' '$16,995']
['150,000km' '70,000km' '50,000km' '80,000km' '10,000km' '30,000km'
 '125,000km' '90,000km' '20,000km' '60,000km' '5,000km' '100,000km'
 '40,000km']
[0]


**We can see that first two columns `price` and `odometer` are potentially numeric columns so we can do some things in order to convert this columns to numeric type. Third column have only one value and this column we will drop in our analyses. Let's make sure, that now  these columns have the type `object`.**

### Convert columns into numeric type

First we convert `price` column:

In [10]:
autos["price"] = autos["price"].str.replace("$", "").str.replace(",","")
print(autos["price"].unique())
autos["price"] = autos["price"].astype(int)
print(autos["price"].dtype)

['5000' '8500' '8990' ... '385' '22200' '16995']
int32


Second we convert `odometer` column and rename the column to `odometer_km`:

In [11]:
autos["odometer"] = autos["odometer"].str.replace("km","").str.replace(",", "")
autos["odometer"] = autos["odometer"].astype(int)
autos.rename({"odometer" : "odometer_km"}, axis = 1, inplace = True)
print(autos["odometer_km"].unique())
print(autos["odometer_km"].dtype)

[150000  70000  50000  80000  10000  30000 125000  90000  20000  60000
   5000 100000  40000]
int32


And let's see the whole dataset after our changes:

In [12]:
print(autos["price"].head())
print(autos["odometer_km"].head())
autos.describe(include = "all")

0    5000
1    8500
2    8990
3    4350
4    1350
Name: price, dtype: int32
0    150000
1    150000
2     70000
3     70000
4    150000
Name: odometer_km, dtype: int32


,date_crawled,name,seller,offer_type,price,abtest,type_vehicle,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,5.000000e+04,50000,44905,50000.000000,47320,50000.000000,47242,50000.000000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,NaN,2,8,NaN,2,NaN,245,NaN,NaN,7,40,2,76,NaN,NaN,39481
top,2016-04-02 11:37:04,Ford_Fiesta,privat,Angebot,NaN,test,limousine,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,NaN,25756,12859,NaN,36993,NaN,4024,NaN,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,9.840044e+03,NaN,NaN,2005.073280,NaN,116.355920,NaN,125732.700000,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,4.811044e+05,NaN,NaN,105.712813,NaN,209.216627,NaN,40042.211706,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,1.100000e+03,NaN,NaN,1999.000000,NaN,70.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,2.950000e+03,NaN,NaN,2003.000000,NaN,105.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,7.200000e+03,NaN,NaN,2008.000000,NaN,150.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


## Exploring the Odometer and Price Columns

Let's explore more complex the data in `odometer` column:

In [13]:
autos["odometer_km"].describe()

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

As we can see, the column has a good values, because there is not any negative or null values. The min is 5000, which is possible for the situation, when the car are resaled. The max value is 150000 which is possible for old cars.

In [14]:
autos["odometer_km"].value_counts()

odometer_km
150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
30000       789
20000       784
10000       264
Name: count, dtype: int64

The most frequint value is 150000, that mean that most of the cars sailed on eBay are old. The least frequint value is 10000, and on other low position are new cars. So we may make a decision, that eBay in common offers the old cars.

So let's explore data in next column - `price`:

In [15]:
autos["price"].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

We can see that the min value in price column are 0, which is impossible in our time. So we need to remove this values from dataset.

And let's explore more with `unique()` method:

In [16]:
autos["price"].value_counts().sort_index(ascending = True)

price
0           1421
1            156
2              3
3              1
5              2
            ... 
10000000       1
11111111       2
12345678       3
27322222       1
99999999       1
Name: count, Length: 2357, dtype: int64

**We can see that the min value in price column are 0, which is impossible in our time as the price lower than 100 dollars. Also we can see that the highest prices are impossible so we can filtered our dataset. Let's determine some bound values:**

**`lower bound` - 100 dollars**

**`upper bound` - 1,000,000 dollars**

In [17]:
autos = autos[(autos["price"] >= 100) & (autos["price"] <= 1000000)]

Let's explore the data in column again:

In [18]:
autos["price"].describe()

count     48227.000000
mean       5992.208099
std       11994.952272
min         100.000000
25%        1250.000000
50%        3000.000000
75%        7499.000000
max      999999.000000
Name: price, dtype: float64

**I make this decision because I think that there is not any car that costs lower than 100 dollars, adn the cars which price is more than 1,000,000 dollars usually sailes in other services and platforms, but not on eBay.**

## Exploring the Date Columns

Right now, the `date_crawled`, `last_seen`, and `ad_created` columns are all identified as string values by pandas. Because these three columns are represented as strings, we need to convert the data into a numerical representation so we can understand it quantitatively.

Firstly, we  calculate the distribution of values in each column:

`date_crawled` column:

In [19]:
only_date = autos["date_crawled"].str[:10]
result = only_date.value_counts(normalize = True, dropna = False).sort_index()
print(result)

date_crawled
2016-03-05    0.025359
2016-03-06    0.014038
2016-03-07    0.036059
2016-03-08    0.033176
2016-03-09    0.033011
2016-03-10    0.032285
2016-03-11    0.032596
2016-03-12    0.036909
2016-03-13    0.015676
2016-03-14    0.036660
2016-03-15    0.034317
2016-03-16    0.029465
2016-03-17    0.031518
2016-03-18    0.012897
2016-03-19    0.034732
2016-03-20    0.037800
2016-03-21    0.037220
2016-03-22    0.032886
2016-03-23    0.032285
2016-03-24    0.029444
2016-03-25    0.031497
2016-03-26    0.032306
2016-03-27    0.031124
2016-03-28    0.034960
2016-03-29    0.034130
2016-03-30    0.033736
2016-03-31    0.031849
2016-04-01    0.033695
2016-04-02    0.035602
2016-04-03    0.038609
2016-04-04    0.036536
2016-04-05    0.013063
2016-04-06    0.003172
2016-04-07    0.001389
Name: proportion, dtype: float64


We can see that our date range is almost a month, with the first date being March 5 and the last date being April 7. The frequency of each date is approximetely equal.

`ad_created` column:

In [20]:
only_date = autos["ad_created"].str[:10]
result = only_date.value_counts(normalize = True, dropna = False).sort_index()
print(result)

ad_created
2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
                ...   
2016-04-03    0.038858
2016-04-04    0.036888
2016-04-05    0.011798
2016-04-06    0.003255
2016-04-07    0.001244
Name: proportion, Length: 76, dtype: float64


We can see that our date range in this column more advanced compared to previous column and is 10 month, with the first date being June 11 and the last 
date being April 7. The frequency of each date is not equal, and has more different values compared to in previous column. The first dates have less frequancy then last dates.

`last_seen` column:

In [21]:
only_date = autos["last_seen"].str[:10]
result = only_date.value_counts(normalize = True, dropna = False).sort_index()
print(result)

last_seen
2016-03-05    0.001078
2016-03-06    0.004313
2016-03-07    0.005433
2016-03-08    0.007320
2016-03-09    0.009580
2016-03-10    0.010637
2016-03-11    0.012400
2016-03-12    0.023783
2016-03-13    0.008875
2016-03-14    0.012628
2016-03-15    0.015862
2016-03-16    0.016443
2016-03-17    0.028096
2016-03-18    0.007320
2016-03-19    0.015759
2016-03-20    0.020652
2016-03-21    0.020549
2016-03-22    0.021357
2016-03-23    0.018579
2016-03-24    0.019761
2016-03-25    0.019097
2016-03-26    0.016671
2016-03-27    0.015551
2016-03-28    0.020860
2016-03-29    0.022311
2016-03-30    0.024696
2016-03-31    0.023825
2016-04-01    0.022850
2016-04-02    0.024882
2016-04-03    0.025131
2016-04-04    0.024530
2016-04-05    0.125054
2016-04-06    0.221971
2016-04-07    0.132146
Name: proportion, dtype: float64


We can see that our date range in this column are not big and almost equal to range of `date_crawled` column and this is expected because the last seen can't have a big range in time and this range is only 1 month , with the first date being March 5 and the last 
date being April 7. As expected, the last dates have big frequency, because every day the most of posts review a lot of persons. The first dates have a small frequancy during the fact, that not many posts are not viewed for weeks.

Let's understand the column `registration_year`:

In [22]:
print(autos["registration_year"].describe())
print("\n")
autos["registration_year"].value_counts()

count    48227.000000
mean      2004.730151
std         87.894768
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64




registration_year
2000    3104
2005    2911
1999    2879
2004    2699
2003    2693
        ... 
1948       1
6200       1
4100       1
4500       1
9000       1
Name: count, Length: 95, dtype: int64

We can see strange values, because the minimum value of year is 1000, which is impossible because there is not any cars in this year.  The maximum value is 9000 which is impossible. And after using `value_counts` method we can observe that there are more incorrect data in this column. So we need fix this problems in the next step.

## Dealing with Incorrect Registration Year Data

So we can determine the highest and lowest values for correct registration year:

- Highest value is 2016, because this is the last year which appears in dataset

- Lowest value is 1920, because I think that in lower years cars are not registered.

We need to remove the values and rows from our dataset, where the values of registration year are outside the range.

So let's do that:

In [23]:
lowest_value = 1920
highest_value = 2016


autos = autos.loc[(autos["registration_year"] >= lowest_value) & (autos["registration_year"] <= highest_value)]

And now we can check the `registration_year` column to determine  the changes:

In [24]:
print(autos["registration_year"].describe())
print("\n")
autos["registration_year"].value_counts().sort_index(ascending = False)

count    46353.000000
mean      2002.943067
std          7.102149
min       1927.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       2016.000000
Name: registration_year, dtype: float64




registration_year
2016    1202
2015     380
2014     662
2013     801
2012    1308
        ... 
1937       4
1934       2
1931       1
1929       1
1927       1
Name: count, Length: 77, dtype: int64

Now we can see that all values are in real range and this years can be the year registration of cars. Good work!

## Exploring Price by Brand

Let's determine some brands, for which we will be calculate the average price.

FIrstly, we need to see all unique brands in `brand` column:

In [25]:
autos["brand"].unique()

array(['peugeot', 'bmw', 'volkswagen', 'smart', 'ford', 'chrysler',
       'seat', 'renault', 'mercedes_benz', 'audi', 'sonstige_autos',
       'opel', 'mazda', 'porsche', 'mini', 'toyota', 'dacia', 'nissan',
       'jeep', 'saab', 'volvo', 'mitsubishi', 'jaguar', 'fiat', 'skoda',
       'subaru', 'kia', 'citroen', 'chevrolet', 'hyundai', 'honda',
       'daewoo', 'suzuki', 'trabant', 'land_rover', 'alfa_romeo', 'lada',
       'rover', 'daihatsu', 'lancia'], dtype=object)

And we want to determine the average price for most popular brands so let's determine it:

In [26]:
most_popular = autos["brand"].value_counts().head(20).index
print(most_popular)

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford', 'renault',
       'peugeot', 'fiat', 'seat', 'skoda', 'nissan', 'mazda', 'smart',
       'citroen', 'toyota', 'hyundai', 'sonstige_autos', 'volvo', 'mini'],
      dtype='object', name='brand')


So now we may do the aggregation with this brands:

In [27]:
avr_price_for_brand = {}

for brand in most_popular:
    mean_value = autos.loc[autos["brand"] == brand, "price"].mean()
    avr_price_for_brand[brand] = mean_value

result = pd.Series(avr_price_for_brand).sort_values(ascending = False)
print(result)

sonstige_autos    12784.556561
mini              10639.450980
audi               9380.718548
mercedes_benz      8672.654241
bmw                8381.677306
skoda              6409.609724
volkswagen         5639.900316
hyundai            5411.075431
toyota             5167.091062
volvo              4993.208038
nissan             4756.659634
seat               4433.419622
mazda              4129.774788
ford               4086.934219
citroen            3796.262673
smart              3596.402736
peugeot            3113.860549
opel               3006.000201
fiat               2836.873631
renault            2496.940394
dtype: float64


**We can see, that the autos, which don't use the oil for his work have the biggest average price, but in future they will be save you a lot of money. The cheapest popular cars are Renault, so if we you want to buy a popular car, but you have not a lot of money, thinking about renault, fiat, opel, or peugeut. My favourite cars are Volkswagen and Mercedes and they have the prices between 5,600 - 9000 dollars.**

## Storing Aggregate Data in a DataFrame

Let's observe the top 6 brands by calculating mean mileage and mean price: 
- Ford and Opel which have low prices.
 
- Mercedes, bmv and Audi, which have high prices.

- Volkswagen, which is the solution between low and high.

In [28]:
top_6_brands = pd.Series(['ford','opel','mercedes_benz','bmw','audi','volkswagen'])

avr_price_for_top_6_brands = {}
avr_mileage_for_top_6_brands = {}

for brand in top_6_brands:
    avr_price = autos.loc[autos["brand"] == brand, "price"].mean()
    avr_mileage = autos.loc[autos["brand"] == brand, "odometer_km"].mean()
    avr_price_for_top_6_brands[brand] = avr_price
    avr_mileage_for_top_6_brands[brand] = avr_mileage

avr_price = pd.Series(avr_price_for_top_6_brands).sort_values(ascending = False)
avr_mileage = pd.Series(avr_mileage_for_top_6_brands).sort_values(ascending = False)


dataframe_avr = pd.DataFrame(avr_price, columns = ["mean_price"])
dataframe_avr["mean_mileage"] = avr_mileage

dataframe_avr.head(6)

,mean_price,mean_mileage
audi,9380.718548,129245.400298
mercedes_benz,8672.654241,131025.669643
bmw,8381.677306,132695.320149
volkswagen,5639.900316,128804.203653
ford,4086.934219,124277.331686
opel,3006.000201,129380.281690


**We can see that the mean mileage are almost equal for all brands, but the price are completely different. So if you have enough money, you will be able tou buy audi, mercedes or bmw. If no, but have enough money for ford and opel, you will buy them. If you have the median, volkswagen is yours.**

# Conclusion

**So in this guided project we practise to clean dataset, some techniques to analyses the data. We also practise to work with pandas.**

**During this project we have done and observe next:**
- Clean and rename column names
- Convert columns dtypes
- Analyse datetime columns
- Analyse other type columns
- Create Series and Dataframes

**We also can continue to analyse our dataset, but our main goal today is already achieve.**